In [38]:
import pandas as pd
import numpy as np

In [87]:
air_filepath = 'data\\ontime.td.201905.asc'

In [88]:
# Load file
air_df = pd.read_csv(air_filepath, sep='|', header=None, dtype=str)
# define titles for the known columns
column_titles = ['carrier_code', 'flight_number', 'unknown', 'unknown', 'unknown', 'unknown',
           'origin_airport', 'destination_airport', 'date', 'unknown',
           'scheduled_departure', 'crs_scheduled_departure', 'actual_departure',
           'scheduled_arrival', 'crs_scheduled_arrival', 'actual_arrival',
           'oag_crs_departure_difference', 'oag_crs_arrival_difference',
           'scheduled_elapsed_time', 'actual_elapsed_time', 'departure_delay', 'arrival_delay',
           'elapsed_time_difference', 'wheels_off_time', 'wheels_on_time', 'tail_number',
           'taxi_out_time', 'taxi_in_time', 'total_in_air_time', 'cancelled_code',
           'delay_carrier', 'delay_weather', 'delay_national_aviation_system', 'delay_security',
           'delay_late_aircarft_arrival', 'unknown', 'actual_elapsed_time_to_diverted_airport', 'unknown',
           'no_diverted_airports', 'diverted_airport_code', 'wheels_on_diverted_airport',
           'total_time_away_gate_diverted_airport', 'longest_period_away_gate_diverted_airport',
           'wheels_off_diverted_airport']
# set column titles
air_df.columns = column_titles + ['unknown' for _ in range(len(air_df.columns) - len(column_titles))]
# keep useful columns
air_df = air_df[['carrier_code', 'flight_number', 'origin_airport', 'destination_airport', 'date', 
                'scheduled_departure', 'actual_departure', 'scheduled_arrival', 'actual_arrival',
               'scheduled_elapsed_time', 'tail_number', 
               'departure_delay', 'arrival_delay', 'delay_carrier', 'delay_weather', 'delay_national_aviation_system', 
               'delay_security', 'delay_late_aircarft_arrival', 'cancelled_code']]

In [89]:
# split date column into 4 columns: year, month, day and weekday
import datetime

air_df['date'] = pd.to_datetime(air_df['date'])
air_df['year'] = air_df['date'].dt.strftime('%Y')
air_df['month'] = air_df['date'].dt.strftime('%m')
air_df['day'] = air_df['date'].dt.strftime('%d')
air_df['weekday'] = air_df['date'].dt.dayofweek
air_df['date'] = air_df['date'].dt.date
# air_df = air_df.drop(columns = ['date'])

In [90]:
def split_time(air_df, column_name):
    air_df[column_name] = air_df[column_name].astype(str).replace('0', np.nan).str.zfill(4).str.replace('^24', '00', regex=True) 
    air_df[column_name] = pd.to_datetime(air_df[column_name], format='%H%M')
    if 'arrival' in column_name:
        air_df['extra'] = pd.to_timedelta((air_df['scheduled_departure_dt'].dt.time > air_df[column_name].dt.time)
                                      .map({False: 0, True: 1}), unit='d')
        air_df[column_name + '_dt'] = pd.to_datetime((air_df.date + air_df['extra']).astype(str) + 
                                                     ' ' + air_df[column_name].dt.time.astype(str), errors='coerce')
        air_df = air_df.drop(columns=['extra'])
    else:
        air_df[column_name + '_dt'] = pd.to_datetime(air_df.date.astype(str) + 
                                                     ' ' + air_df[column_name].dt.time.astype(str), errors='coerce')
    air_df = air_df.drop(columns=[column_name])
    return air_df
air_df = split_time(air_df, 'scheduled_departure')
air_df = split_time(air_df, 'scheduled_arrival')
air_df = split_time(air_df, 'actual_departure')
air_df = split_time(air_df, 'actual_arrival')

In [91]:
# fill NaN values in cancelled_code column with 0
air_df.cancelled_code = air_df.cancelled_code.fillna(value='N')

In [92]:
air_df.to_csv('ontime_05_2019.csv', index=False)

In [93]:
air_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694336 entries, 0 to 694335
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   carrier_code                    694336 non-null  object        
 1   flight_number                   694336 non-null  object        
 2   origin_airport                  694336 non-null  object        
 3   destination_airport             694336 non-null  object        
 4   date                            694336 non-null  object        
 5   scheduled_elapsed_time          694336 non-null  object        
 6   tail_number                     692586 non-null  object        
 7   departure_delay                 694336 non-null  object        
 8   arrival_delay                   694336 non-null  object        
 9   delay_carrier                   694336 non-null  object        
 10  delay_weather                   694336 non-null  object 

In [94]:
import glob, os
import pandas as pd

In [95]:
wdf = pd.concat(map(pd.read_csv, glob.glob(os.path.join('data', 'weather-data', '2019', "*.csv"))))
wdf['DATE'] = pd.to_datetime(wdf['DATE'])

In [96]:
wdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3952678 entries, 0 to 8739
Data columns (total 8 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   STATION                   object        
 1   DATE                      datetime64[ns]
 2   HourlyDryBulbTemperature  float64       
 3   HourlyPrecipitation       float64       
 4   HourlyStationPressure     float64       
 5   HourlyVisibility          float64       
 6   HourlyWindSpeed           float64       
 7   iata_code                 object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 271.4+ MB


In [97]:
def add_weather_data(air_df, wdf, scheduled_col, airport_col):
    air_df_s = air_df.rename(columns={scheduled_col: 'DATE'}).sort_values(by=['DATE'])
    wdf_s = wdf.rename(columns={'iata_code': airport_col}).sort_values(by=['DATE'])
    merged = pd.merge_asof(air_df_s, wdf_s, on='DATE', by=airport_col, direction='nearest')
    merged = merged.rename(columns={'DATE': scheduled_col})
    return merged

In [98]:
merged = add_weather_data(air_df, wdf, 'scheduled_departure_dt', 'origin_airport')
merged = add_weather_data(merged, wdf, 'scheduled_arrival_dt', 'destination_airport')
# merged = merged.drop(columns=['date', 'STATION'])
merged.to_csv('merged-05-2019.csv', index=False)

In [27]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655104 entries, 0 to 655103
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   carrier_code                    655104 non-null  object        
 1   flight_number                   655104 non-null  object        
 2   origin_airport                  655104 non-null  object        
 3   destination_airport             655104 non-null  object        
 4   date                            655104 non-null  object        
 5   scheduled_elapsed_time          655104 non-null  object        
 6   tail_number                     653795 non-null  object        
 7   departure_delay                 655104 non-null  object        
 8   arrival_delay                   655104 non-null  object        
 9   delay_carrier                   655104 non-null  object        
 10  delay_weather                   655104 non-null  object 

In [28]:
merged['HourlyPrecipitation_x'].fillna(method='ffill')

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
655099    0.0
655100    0.0
655101    0.0
655102    0.0
655103    0.0
Name: HourlyPrecipitation_x, Length: 655104, dtype: float64